In [14]:
from tdc.single_pred import Tox
import deepchem
import xgboost
import rdkit
from rdkit import Chem
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem.EState import EState

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

Found local copy...


In [35]:
benchmark = group.get('LD50_Zhu') 
name = benchmark['name']
train_val, test = benchmark['train_val'], benchmark['test']

firstMol = Chem.MolFromSmiles(train_val.iloc[:,1].to_list()[0])

np.array(EState.EStateIndices(firstMol))

#Fingerprinter.FingerprintMol(firstMol)

KeyError: 0

In [32]:
predictions_list = []
metric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.mean_absolute_error)
xgb_reg = xgboost.XGBRegressor()

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('LD50_Zhu') 
    
    predictions = {}
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)  

    trainMol = train.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    validMol = valid.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    testMol = test.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))

    #featurize training, valid, and test data for xgboost
    trainFeat = trainMol.map(lambda x: np.array(EState.EStateIndices(x))).to_numpy()
    validFeat = np.array(validMol.map(lambda x: EState.EStateIndices(x)).to_list())
    testFeat = np.array(testMol.map(lambda x: EState.EStateIndices(x)).to_list())

    print(trainMol)
    print(trainFeat)
    #valid_data = list(zip(ecfp_f_valid, valid.iloc[:,2].to_list()))
    #valid_data = [(ecfp_f_train, train.iloc[:,2].to_list()), (ecfp_f_valid, valid.iloc[:,2].to_list())]

    #fit xgboost model and store np ndarray in xgb_pred
    xgb_reg.fit(X=trainFeat, y=valid.iloc[:,2], eval_metric="mae", verbose=True)
    xgb_pred = xgb_reg.predict(X=testFeat)

    # store test predictions in y_pred_test
    y_pred_test = xgb_pred

    print("predictions for run #")
    print(seed)
    print(y_pred_test[0:5])
        
    predictions[name] = y_pred_test
    predictions_list.append(predictions)

print("Prediction List:")
print(predictions_list)

generating training, validation splits...
100%|██████████| 5907/5907 [00:01<00:00, 3905.60it/s]


0       <rdkit.Chem.rdchem.Mol object at 0x000002086CD...
1       <rdkit.Chem.rdchem.Mol object at 0x000002085F8...
2       <rdkit.Chem.rdchem.Mol object at 0x000002085F3...
3       <rdkit.Chem.rdchem.Mol object at 0x000002085F3...
4       <rdkit.Chem.rdchem.Mol object at 0x000002086CD...
                              ...                        
5163    <rdkit.Chem.rdchem.Mol object at 0x000002086CE...
5164    <rdkit.Chem.rdchem.Mol object at 0x000002086CE...
5165    <rdkit.Chem.rdchem.Mol object at 0x000002086CE...
5166    <rdkit.Chem.rdchem.Mol object at 0x000002086CE...
5167    <rdkit.Chem.rdchem.Mol object at 0x000002086CE...
Name: Drug, Length: 5168, dtype: object
[array([11.61266188, -0.26247118,  5.19306595,  0.41442555,  0.74941823,
         1.17440896,  2.02416005,  1.98050642,  1.97170514,  1.98050642,
         2.02416005,  0.59774229,  1.75481198,  1.82717167,  1.87574292,
         1.82717167,  1.75481198])
 array([ 1.74343085,  0.28714517,  5.00647944, -0.44803971, 11.92939

ValueError: Please reshape the input data into 2-dimensional matrix.